Compute the KS test to determine whether the void and wall samples are drawn from the same or different parent distributions.

Also compute the Wilcoxon rank-sum test to determine if the void or wall sample is stochastically larger than the other.

In [2]:
from astropy.table import Table
from astropy.io import fits

import numpy as np

import sys

import pickle

import dynesty
from dynesty import plotting as dyplot

import matplotlib
import matplotlib.pyplot as plt

#from functions import Model_1_fit, Model_2_fit, Model_1_plot, Model_2_plot, Model_1_sampler, Model_2_sampler, uniform, jeffreys

from scipy.stats import ks_2samp, ranksums
#from scipy import stats
#from scipy.optimize import curve_fit
#from scipy.stats import skewnorm

np.set_printoptions(threshold=sys.maxsize)

In [3]:
matplotlib.rc('font', size=14)
matplotlib.rc('font', family='DejaVu Sans')

# Data

In [4]:
data_filename = '../../../../data/NSA_v1_0_1_VAGC_vflag-V2-VF.fits'

hdu = fits.open(data_filename)
data = Table(hdu[1].data)
hdu.close()

## Just keep the main SDSS DR7 footprint

In [5]:
# When the catalog is the NSA, only keep those objects which were originally in the DR7 LSS
catalog_SDSS = data[data['IN_DR7_LSS'] == 1]

ra_boolean = np.logical_and(catalog_SDSS['RA'] > 110, catalog_SDSS['RA'] < 270)
catalog_north = catalog_SDSS[ra_boolean]

strip_boolean = np.logical_and.reduce([catalog_north['RA'] > 250, 
                                       catalog_north['RA'] < 269, 
                                       catalog_north['DEC'] > 51, 
                                       catalog_north['DEC'] < 67])
catalog_main = catalog_north[~strip_boolean]

In [6]:
################################################################################
# stellar mass
#-------------------------------------------------------------------------------
Mstar_NSA = np.log10(catalog_main['ELPETRO_MASS'])
################################################################################


################################################################################
# color
#-------------------------------------------------------------------------------
u_r_NSA = np.array(catalog_main['u_r'])

g_r_NSA = np.array(catalog_main['g_r'])
################################################################################


################################################################################
# absolute magnitude
#-------------------------------------------------------------------------------
rabsmag_NSA = np.array(catalog_main['ELPETRO_ABSMAG'][:,4])
################################################################################


################################################################################
# star formation rate and specific SPR
#-------------------------------------------------------------------------------
SFR  = np.array(catalog_main['SFR'])

sSFR = np.array(catalog_main['sSFR'])
################################################################################

/var/folders/5x/2533z9ns34n_8ghxd5f6hl0c0000gn/T/ipykernel_4401/2049853064.py:4: RuntimeWarning: divide by zero encountered in log10
  Mstar_NSA = np.log10(catalog_main['ELPETRO_MASS'])


## Separate galaxies by their LSS classifications

### $V^2$

In [7]:
wall_v2 = catalog_main['vflag_V2'] == 0
void_v2 = catalog_main['vflag_V2'] == 1
edge_v2 = catalog_main['vflag_V2'] == 2
out_v2 = catalog_main['vflag_V2'] == 9

### VoidFinder

In [8]:
wall_vf = catalog_main['vflag_VF'] == 0
void_vf = catalog_main['vflag_VF'] == 1
edge_vf = catalog_main['vflag_VF'] == 2
out_vf = catalog_main['vflag_VF'] == 9

# Calculate the KS test statistic and associated $p$-value

## $M_*$

In [9]:
VF_Mstar_ks_stat, VF_Mstar_p = ks_2samp(Mstar_NSA[void_vf], Mstar_NSA[wall_vf])

V2_Mstar_ks_stat, V2_Mstar_p = ks_2samp(Mstar_NSA[void_v2], Mstar_NSA[wall_v2])

print('VF KS statistic = {:.3f}    p-value = {:.3e}'.format(VF_Mstar_ks_stat, VF_Mstar_p))
print('V2 KS statistic = {:.3f}    p-value = {:.3e}'.format(V2_Mstar_ks_stat, V2_Mstar_p))

VF KS statistic = 0.134    p-value = 0.000e+00
V2 KS statistic = 0.040    p-value = 4.539e-99


In [10]:
VF_Mstar_wilcoxon_stat, VF_Mstar_wilcoxon_p = ranksums(Mstar_NSA[void_vf], Mstar_NSA[wall_vf])#, nan_policy='omit')

V2_Mstar_wilcoxon_stat, V2_Mstar_wilcoxon_p = ranksums(Mstar_NSA[void_v2], Mstar_NSA[wall_v2])#, nan_policy='omit')

print('One of the two is stochastically greater than the other.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_Mstar_wilcoxon_stat, VF_Mstar_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_Mstar_wilcoxon_stat, V2_Mstar_wilcoxon_p))

One of the two is stochastically greater than the other.
VF Wilcoxon statistic = -83.197    p-value = 0.000e+00
V2 Wilcoxon statistic = -22.358    p-value = 1.014e-110


In [11]:
VF_Mstar_wilcoxon_stat, VF_Mstar_wilcoxon_p = ranksums(Mstar_NSA[void_vf], 
                                                       Mstar_NSA[wall_vf], 
                                                       alternative='less')#, nan_policy='omit')

V2_Mstar_wilcoxon_stat, V2_Mstar_wilcoxon_p = ranksums(Mstar_NSA[void_v2], 
                                                       Mstar_NSA[wall_v2], 
                                                       alternative='less')#, nan_policy='omit')

print('Void is stochastically less than wall.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_Mstar_wilcoxon_stat, VF_Mstar_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_Mstar_wilcoxon_stat, V2_Mstar_wilcoxon_p))

Void is stochastically less than wall.
VF Wilcoxon statistic = -83.197    p-value = 0.000e+00
V2 Wilcoxon statistic = -22.358    p-value = 5.071e-111


In [12]:
VF_Mstar_wilcoxon_stat, VF_Mstar_wilcoxon_p = ranksums(Mstar_NSA[void_vf], 
                                                       Mstar_NSA[wall_vf], 
                                                       alternative='greater')#, nan_policy='omit')

V2_Mstar_wilcoxon_stat, V2_Mstar_wilcoxon_p = ranksums(Mstar_NSA[void_v2], 
                                                       Mstar_NSA[wall_v2], 
                                                       alternative='greater')#, nan_policy='omit')

print('Void is stochastically greater than wall.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_Mstar_wilcoxon_stat, VF_Mstar_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_Mstar_wilcoxon_stat, V2_Mstar_wilcoxon_p))

Void is stochastically greater than wall.
VF Wilcoxon statistic = -83.197    p-value = 1.000e+00
V2 Wilcoxon statistic = -22.358    p-value = 1.000e+00


## $M_r$

In [13]:
VF_rabsmag_ks_stat, VF_rabsmag_p = ks_2samp(rabsmag_NSA[void_vf], rabsmag_NSA[wall_vf])

V2_rabsmag_ks_stat, V2_rabsmag_p = ks_2samp(rabsmag_NSA[void_v2], rabsmag_NSA[wall_v2])

print('VF KS statistic = {:.3f}    p-value = {:.3e}'.format(VF_rabsmag_ks_stat, VF_rabsmag_p))
print('V2 KS statistic = {:.3f}    p-value = {:.3e}'.format(V2_rabsmag_ks_stat, V2_rabsmag_p))

VF KS statistic = 0.118    p-value = 0.000e+00
V2 KS statistic = 0.042    p-value = 9.312e-110


In [14]:
VF_rabsmag_wilcoxon_stat, VF_rabsmag_wilcoxon_p = ranksums(rabsmag_NSA[void_vf], 
                                                           rabsmag_NSA[wall_vf])#, nan_policy='omit')

V2_rabsmag_wilcoxon_stat, V2_rabsmag_wilcoxon_p = ranksums(rabsmag_NSA[void_v2], 
                                                           rabsmag_NSA[wall_v2])#, nan_policy='omit')

print('One of the two is stochastically greater than the other.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_rabsmag_wilcoxon_stat, VF_rabsmag_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_rabsmag_wilcoxon_stat, V2_rabsmag_wilcoxon_p))

One of the two is stochastically greater than the other.
VF Wilcoxon statistic = 69.823    p-value = 0.000e+00
V2 Wilcoxon statistic = 23.826    p-value = 1.803e-125


In [15]:
VF_rabsmag_wilcoxon_stat, VF_rabsmag_wilcoxon_p = ranksums(rabsmag_NSA[void_vf], 
                                                           rabsmag_NSA[wall_vf], 
                                                           alternative='less')#, nan_policy='omit')

V2_rabsmag_wilcoxon_stat, V2_rabsmag_wilcoxon_p = ranksums(rabsmag_NSA[void_v2], 
                                                           rabsmag_NSA[wall_v2], 
                                                           alternative='less')#, nan_policy='omit')

print('Void is stochastically less than wall.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_rabsmag_wilcoxon_stat, VF_rabsmag_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_rabsmag_wilcoxon_stat, V2_rabsmag_wilcoxon_p))

Void is stochastically less than wall.
VF Wilcoxon statistic = 69.823    p-value = 1.000e+00
V2 Wilcoxon statistic = 23.826    p-value = 1.000e+00


In [16]:
VF_rabsmag_wilcoxon_stat, VF_rabsmag_wilcoxon_p = ranksums(rabsmag_NSA[void_vf], 
                                                           rabsmag_NSA[wall_vf], 
                                                           alternative='greater')#, nan_policy='omit')

V2_rabsmag_wilcoxon_stat, V2_rabsmag_wilcoxon_p = ranksums(rabsmag_NSA[void_v2], 
                                                           rabsmag_NSA[wall_v2], 
                                                           alternative='greater')#, nan_policy='omit')

print('Void is stochastically greater than wall.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_rabsmag_wilcoxon_stat, VF_rabsmag_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_rabsmag_wilcoxon_stat, V2_rabsmag_wilcoxon_p))

Void is stochastically greater than wall.
VF Wilcoxon statistic = 69.823    p-value = 0.000e+00
V2 Wilcoxon statistic = 23.826    p-value = 9.014e-126


## Color

### $u - r$

In [17]:
VF_ur_ks_stat, VF_ur_p = ks_2samp(u_r_NSA[void_vf], u_r_NSA[wall_vf])

V2_ur_ks_stat, V2_ur_p = ks_2samp(u_r_NSA[void_v2], u_r_NSA[wall_v2])

print('VF KS statistic = {:.3f}    p-value = {:.3e}'.format(VF_ur_ks_stat, VF_ur_p))
print('V2 KS statistic = {:.3f}    p-value = {:.3e}'.format(V2_ur_ks_stat, V2_ur_p))

VF KS statistic = 0.148    p-value = 0.000e+00
V2 KS statistic = 0.020    p-value = 2.885e-26


In [18]:
VF_ur_wilcoxon_stat, VF_ur_wilcoxon_p = ranksums(u_r_NSA[void_vf], 
                                                 u_r_NSA[wall_vf])#, nan_policy='omit')

V2_ur_wilcoxon_stat, V2_ur_wilcoxon_p = ranksums(u_r_NSA[void_v2], 
                                                 u_r_NSA[wall_v2])#, nan_policy='omit')

print('One of the two is stochastically greater than the other.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_ur_wilcoxon_stat, VF_ur_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_ur_wilcoxon_stat, V2_ur_wilcoxon_p))

One of the two is stochastically greater than the other.
VF Wilcoxon statistic = -84.308    p-value = 0.000e+00
V2 Wilcoxon statistic = -13.024    p-value = 8.953e-39


In [19]:
VF_ur_wilcoxon_stat, VF_ur_wilcoxon_p = ranksums(u_r_NSA[void_vf], 
                                                 u_r_NSA[wall_vf], 
                                                 alternative='less')#, nan_policy='omit')

V2_ur_wilcoxon_stat, V2_ur_wilcoxon_p = ranksums(u_r_NSA[void_v2], 
                                                 u_r_NSA[wall_v2], 
                                                 alternative='less')#, nan_policy='omit')

print('Void is stochastically less than wall.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_ur_wilcoxon_stat, VF_ur_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_ur_wilcoxon_stat, V2_ur_wilcoxon_p))

Void is stochastically less than wall.
VF Wilcoxon statistic = -84.308    p-value = 0.000e+00
V2 Wilcoxon statistic = -13.024    p-value = 4.477e-39


In [20]:
VF_ur_wilcoxon_stat, VF_ur_wilcoxon_p = ranksums(u_r_NSA[void_vf], 
                                                 u_r_NSA[wall_vf], 
                                                 alternative='greater')#, nan_policy='omit')

V2_ur_wilcoxon_stat, V2_ur_wilcoxon_p = ranksums(u_r_NSA[void_v2], 
                                                 u_r_NSA[wall_v2], 
                                                 alternative='greater')#, nan_policy='omit')

print('Void is stochastically greater than wall.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_ur_wilcoxon_stat, VF_ur_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3f}'.format(V2_ur_wilcoxon_stat, V2_ur_wilcoxon_p))

Void is stochastically greater than wall.
VF Wilcoxon statistic = -84.308    p-value = 1.000e+00
V2 Wilcoxon statistic = -13.024    p-value = 1.000


### $g - r$

In [21]:
VF_gr_ks_stat, VF_gr_p = ks_2samp(g_r_NSA[void_vf], g_r_NSA[wall_vf])

V2_gr_ks_stat, V2_gr_p = ks_2samp(g_r_NSA[void_v2], g_r_NSA[wall_v2])

print('VF KS statistic = {:.3f}    p-value = {:.3e}'.format(VF_gr_ks_stat, VF_gr_p))
print('V2 KS statistic = {:.3f}    p-value = {:.3e}'.format(V2_gr_ks_stat, V2_gr_p))

VF KS statistic = 0.150    p-value = 0.000e+00
V2 KS statistic = 0.027    p-value = 5.645e-47


In [22]:
VF_gr_wilcoxon_stat, VF_gr_wilcoxon_p = ranksums(g_r_NSA[void_vf], 
                                                 g_r_NSA[wall_vf])#, nan_policy='omit')

V2_gr_wilcoxon_stat, V2_gr_wilcoxon_p = ranksums(g_r_NSA[void_v2], 
                                                 g_r_NSA[wall_v2])#, nan_policy='omit')

print('One of the two is stochastically greater than the other.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_gr_wilcoxon_stat, VF_gr_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_gr_wilcoxon_stat, V2_gr_wilcoxon_p))

One of the two is stochastically greater than the other.
VF Wilcoxon statistic = -85.948    p-value = 0.000e+00
V2 Wilcoxon statistic = -16.555    p-value = 1.480e-61


In [23]:
VF_gr_wilcoxon_stat, VF_gr_wilcoxon_p = ranksums(g_r_NSA[void_vf], 
                                                 g_r_NSA[wall_vf], 
                                                 alternative='less')#, nan_policy='omit')

V2_gr_wilcoxon_stat, V2_gr_wilcoxon_p = ranksums(g_r_NSA[void_v2], 
                                                 g_r_NSA[wall_v2], 
                                                 alternative='less')#, nan_policy='omit')

print('Void is stochastically less than wall.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_gr_wilcoxon_stat, VF_gr_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_gr_wilcoxon_stat, V2_gr_wilcoxon_p))

Void is stochastically less than wall.
VF Wilcoxon statistic = -85.948    p-value = 0.000e+00
V2 Wilcoxon statistic = -16.555    p-value = 7.400e-62


In [24]:
VF_gr_wilcoxon_stat, VF_gr_wilcoxon_p = ranksums(g_r_NSA[void_vf], 
                                                 g_r_NSA[wall_vf], 
                                                 alternative='greater')#, nan_policy='omit')

V2_gr_wilcoxon_stat, V2_gr_wilcoxon_p = ranksums(g_r_NSA[void_v2], 
                                                 g_r_NSA[wall_v2], 
                                                 alternative='greater')#, nan_policy='omit')

print('Void is stochastically greater than wall.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_gr_wilcoxon_stat, VF_gr_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_gr_wilcoxon_stat, V2_gr_wilcoxon_p))

Void is stochastically greater than wall.
VF Wilcoxon statistic = -85.948    p-value = 1.000e+00
V2 Wilcoxon statistic = -16.555    p-value = 1.000e+00


## SFR

In [25]:
VF_SFR_ks_stat, VF_SFR_p = ks_2samp(SFR[void_vf], SFR[wall_vf])

V2_SFR_ks_stat, V2_SFR_p = ks_2samp(SFR[void_v2], SFR[wall_v2])

print('VF KS statistic = {:.3f}    p-value = {:.3e}'.format(VF_SFR_ks_stat, VF_SFR_p))
print('V2 KS statistic = {:.3f}    p-value = {:.3e}'.format(V2_SFR_ks_stat, V2_SFR_p))

VF KS statistic = 0.088    p-value = 0.000e+00
V2 KS statistic = 0.023    p-value = 1.672e-34


In [29]:
VF_SFR_wilcoxon_stat, VF_SFR_wilcoxon_p = ranksums(SFR[void_vf], 
                                                   SFR[wall_vf])#, nan_policy='omit')

V2_SFR_wilcoxon_stat, V2_SFR_wilcoxon_p = ranksums(SFR[void_v2], 
                                                   SFR[wall_v2])#, nan_policy='omit')

print('One of the two is stochastically greater than the other.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_SFR_wilcoxon_stat, VF_SFR_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_SFR_wilcoxon_stat, V2_SFR_wilcoxon_p))

One of the two is stochastically greater than the other.
VF Wilcoxon statistic = 44.075    p-value = 0.000e+00
V2 Wilcoxon statistic = -4.975    p-value = 6.535e-07


In [30]:
VF_SFR_wilcoxon_stat, VF_SFR_wilcoxon_p = ranksums(SFR[void_vf], 
                                                   SFR[wall_vf], 
                                                   alternative='less')#, nan_policy='omit')

V2_SFR_wilcoxon_stat, V2_SFR_wilcoxon_p = ranksums(SFR[void_v2], 
                                                   SFR[wall_v2], 
                                                   alternative='less')#, nan_policy='omit')

print('Void is stochastically less than wall.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_SFR_wilcoxon_stat, VF_SFR_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_SFR_wilcoxon_stat, V2_SFR_wilcoxon_p))

Void is stochastically less than wall.
VF Wilcoxon statistic = 44.075    p-value = 1.000e+00
V2 Wilcoxon statistic = -4.975    p-value = 3.267e-07


In [31]:
VF_SFR_wilcoxon_stat, VF_SFR_wilcoxon_p = ranksums(SFR[void_vf], 
                                                   SFR[wall_vf], 
                                                   alternative='greater')#, nan_policy='omit')

V2_SFR_wilcoxon_stat, V2_SFR_wilcoxon_p = ranksums(SFR[void_v2], 
                                                   SFR[wall_v2], 
                                                   alternative='greater')#, nan_policy='omit')

print('Void is stochastically greater than wall.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_SFR_wilcoxon_stat, VF_SFR_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_SFR_wilcoxon_stat, V2_SFR_wilcoxon_p))

Void is stochastically greater than wall.
VF Wilcoxon statistic = 44.075    p-value = 0.000e+00
V2 Wilcoxon statistic = -4.975    p-value = 1.000e+00


## sSFR

In [33]:
VF_sSFR_ks_stat, VF_sSFR_p = ks_2samp(sSFR[void_vf], sSFR[wall_vf])

V2_sSFR_ks_stat, V2_sSFR_p = ks_2samp(sSFR[void_v2], sSFR[wall_v2])

print('VF KS statistic = {:.3f}    p-value = {:.3e}'.format(VF_sSFR_ks_stat, VF_sSFR_p))
print('V2 KS statistic = {:.3f}    p-value = {:.3e}'.format(V2_sSFR_ks_stat, V2_sSFR_p))

VF KS statistic = 0.144    p-value = 0.000e+00
V2 KS statistic = 0.019    p-value = 4.159e-23


In [34]:
VF_sSFR_wilcoxon_stat, VF_sSFR_wilcoxon_p = ranksums(sSFR[void_vf], 
                                                     sSFR[wall_vf])#, nan_policy='omit')

V2_sSFR_wilcoxon_stat, V2_sSFR_wilcoxon_p = ranksums(sSFR[void_v2], 
                                                     sSFR[wall_v2])#, nan_policy='omit')

print('One of the two is stochastically greater than the other.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_sSFR_wilcoxon_stat, VF_sSFR_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_sSFR_wilcoxon_stat, V2_sSFR_wilcoxon_p))

One of the two is stochastically greater than the other.
VF Wilcoxon statistic = 80.796    p-value = 0.000e+00
V2 Wilcoxon statistic = 10.979    p-value = 4.818e-28


In [35]:
VF_sSFR_wilcoxon_stat, VF_sSFR_wilcoxon_p = ranksums(sSFR[void_vf], 
                                                     sSFR[wall_vf], 
                                                     alternative='less')#, nan_policy='omit')

V2_sSFR_wilcoxon_stat, V2_sSFR_wilcoxon_p = ranksums(sSFR[void_v2], 
                                                     sSFR[wall_v2], 
                                                     alternative='less')#, nan_policy='omit')

print('Void is stochastically less than wall.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_sSFR_wilcoxon_stat, VF_sSFR_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_sSFR_wilcoxon_stat, V2_sSFR_wilcoxon_p))

Void is stochastically less than wall.
VF Wilcoxon statistic = 80.796    p-value = 1.000e+00
V2 Wilcoxon statistic = 10.979    p-value = 1.000e+00


In [36]:
VF_sSFR_wilcoxon_stat, VF_sSFR_wilcoxon_p = ranksums(sSFR[void_vf], 
                                                     sSFR[wall_vf], 
                                                     alternative='greater')#, nan_policy='omit')

V2_sSFR_wilcoxon_stat, V2_sSFR_wilcoxon_p = ranksums(sSFR[void_v2], 
                                                     sSFR[wall_v2], 
                                                     alternative='greater')#, nan_policy='omit')

print('Void is stochastically greater than wall.')
print('VF Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(VF_sSFR_wilcoxon_stat, VF_sSFR_wilcoxon_p))
print('V2 Wilcoxon statistic = {:.3f}    p-value = {:.3e}'.format(V2_sSFR_wilcoxon_stat, V2_sSFR_wilcoxon_p))

Void is stochastically greater than wall.
VF Wilcoxon statistic = 80.796    p-value = 0.000e+00
V2 Wilcoxon statistic = 10.979    p-value = 2.409e-28
